In [98]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [99]:
gdp_data = requests.get('https://www.imf.org/external/datamapper/api/v1/NGDPD?periods=2025,2025')
group_data = requests.get('https://www.imf.org/external/datamapper/api/v1/groups')
region_data = requests.get('https://www.imf.org/external/datamapper/api/v1/regions')

gdp_json_obejct = json.loads(gdp_data.text)
group_json_object = json.loads(group_data.text)
region_json_object = json.loads(region_data.text)

In [126]:
with open('./data/gdp.json','w', encoding='utf-8') as gdp_file:
    json.dump(gdp_json_obejct, gdp_file, indent=4)
gdp_file.close()

with open('./data/group.json','w', encoding='utf-8') as group_file:
    json.dump(group_json_object, group_file, indent=4)
group_file.close()

with open('./data/region.json','w', encoding='utf-8') as region_file:
    json.dump(region_json_object, region_file, indent=4)
region_file.close()

gdp_df = pd.DataFrame(gdp_json_obejct['values']['NGDPD'])
group_df = pd.DataFrame(group_json_object['groups'])
region_df = pd.DataFrame(region_json_object['regions'])

In [127]:
gdp_df = pd.melt(gdp_df)
gdp_df.rename(columns={'variable':'Country', 'value':'GDP_USD_billion'},inplace=True)
gdp_df.sort_values(by='GDP_USD_billion',ascending=False, inplace=True)
gdp_df.reset_index(inplace=True, drop=True)
gdp_df

,Country,GDP_USD_billion
0,WEOWORLD,115494.312
1,ADVEC,67561.045
2,MAE,51451.646
3,OEMDC,47933.267
4,APQ,41024.438
...,...,...
216,PLW,0.353
217,KIR,0.333
218,MHL,0.294
219,NRU,0.179


In [128]:
group_df = pd.melt(group_df)
group_df.rename(columns={'variable':'group','value':'mean'}, inplace=True)
group_df

,group,mean
0,ADVEC,Advanced economies
1,AEEUEJ,"Adv econ excl US, Euro, Japan"
2,AFR,Africa (Analytical)
3,AS5,ASEAN-5
4,CEE,Central and Eastern Europe
...,...,...
124,FR_FC_ADV,Advanced Economies
125,FR_FC_EME,Emerging Market and Developing Economies
126,FR_FC_EU,European Union
127,FR_FC_NonEU,Non European Union


In [129]:
region_df = pd.melt(region_df)
region_df.rename(columns={'variable':'region','value':'mean'}, inplace=True)
region_df

,region,mean
0,AFQ,Africa (Region)
1,AFRREO,African Regional Economic Outlook
2,APQ,Asia and Pacific
3,AZQ,Australia and New Zealand
4,CAQ,Central Asia and the Caucasus
5,CBQ,Caribbean
6,CMQ,Central America
7,EAQ,East Asia
8,EEQ,Eastern Europe
9,EUQ,Europe


In [130]:
gdp_df = gdp_df[~gdp_df['Country'].isin(group_df['group'])]
gdp_df = gdp_df[~gdp_df['Country'].isin(region_df['region'])]
gdp_df.reset_index(inplace=True, drop=True)
gdp_df

,Country,GDP_USD_billion
0,USA,30337.162
1,CHN,19534.894
2,DEU,4921.563
3,JPN,4389.326
4,IND,4271.922
...,...,...
184,PLW,0.353
185,KIR,0.333
186,MHL,0.294
187,NRU,0.179


In [131]:
region_info_df = pd.read_csv('./data/region.csv')
gdp_df = gdp_df.merge(
    region_info_df[['alpha-3','region']],
    left_on='Country',
    right_on='alpha-3',
    how='left'
    )
gdp_df.drop(columns='alpha-3', inplace=True)